In [1]:
import warnings
warnings.filterwarnings('ignore')

sys.path.append(os.path.abspath('../../src'))

In [2]:
import numpy as np
import torch

from copolymer_simulator import Simulator
from fixed_noise_gp import get_gp, train_gp

In [3]:
simulator = Simulator()

In [4]:
X1_range = np.linspace(15, 53, 20)
X2_range = np.linspace(0, 100, 51)

X1, X2 = np.meshgrid(X1_range, X2_range)
X1_flat = X1.flatten()
X2_flat = X2.flatten()
grid_points = np.column_stack((X1_flat, X2_flat))  # Shape (1020, 2)
X = torch.from_numpy(grid_points)
X = X.clone().detach().requires_grad_(True)

In [5]:
y = torch.tensor(simulator.data_table.sort_values(by=['x2', 'x1'])['rg_mean']).reshape(-1, 1)
noise = torch.tensor(simulator.data_table.sort_values(by=['x2', 'x1'])['rg_std']).reshape(-1, 1)

In [6]:
model = train_gp(get_gp(X, y, noise))

In [7]:
model.eval()
posterior = model.posterior(X)
mean = posterior.mean.squeeze()

grads = torch.autograd.grad(
    outputs=mean,
    inputs=X,
    grad_outputs=torch.ones_like(mean),
    retain_graph=True,
    create_graph=False,
)[0]

df_dx1 = grads[:, 0]

In [8]:
df_dx1 = df_dx1.reshape((51, 20)).detach().numpy()